# Install Dependencies

In [1]:
# !pip install pytorch_lightning
# !pip install positional_encodings
# !pip install randaugment

In [2]:
import json
import tqdm
import os
import pandas as pd

In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
# file = open("../../Data/FathomNet2023/coco_style/annotations/instances_train2014.json")
# train_metadata = json.load(file)
# train_metadata = pd.DataFrame(train_metadata['images'])

# base_path = "../../Data/FathomNet2023/coco_style/images/train2014/"
# for file in tqdm.tqdm(train_metadata['file_name'], total=len(train_metadata)):
#     try:
#         Image.open(os.path.join(base_path, file))
#     except:
#         print(file)

In [5]:
# file = open("../../Data/FathomNet2023/coco_style/annotations/instances_val2014.json")
# train_metadata = json.load(file)
# train_metadata = pd.DataFrame(train_metadata['images'])

# base_path = "../../Data/FathomNet2023/coco_style/images/val2014/"
# for file in tqdm.tqdm(train_metadata['file_name'], total=len(train_metadata)):
#     try:
#         Image.open(os.path.join(base_path, file))
#     except:
#         print(file)

### Models

In [6]:
import torch
import math
import timm
import torch.nn as nn
from torchvision import models
import torch.nn.functional as F
import pytorch_lightning as pl
import wandb

from torch.optim.lr_scheduler import OneCycleLR
from torchmetrics.functional import accuracy, precision
import torchmetrics.functional as tf
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import TQDMProgressBar
from pytorch_lightning.callbacks import StochasticWeightAveraging
from positional_encodings.torch_encodings import PositionalEncodingPermute2D, Summer

from q2l_labeller.data.coco_data_module import COCODataModule
from q2l_labeller.pl_modules.query2label_train_module import Query2LabelTrainModule
from q2l_labeller.data.cutmix import CutMixCriterion
from q2l_labeller.loss_modules.simple_asymmetric_loss import AsymmetricLoss

In [7]:
class TimmBackbone(nn.Module):
    def __init__(self, model_name):
        super().__init__()

        # Creating the model in this way produces unpooled, unclassified features
        self.model = timm.create_model(
            model_name, 
            pretrained=True,
            num_classes=0,
            global_pool='')        

    def forward(self, x):
        
        out = self.model(x)
        
        return out

In [8]:
class Query2Label(nn.Module):
    """Modified Query2Label model

    Unlike the model described in the paper (which uses a modified DETR 
    transformer), this version uses a standard, unmodified Pytorch Transformer. 
    Learnable label embeddings are passed to the decoder module as the target 
    sequence (and ultimately is passed as the Query to MHA).
    """
    def __init__(
        self, model, conv_out, num_classes, hidden_dim=256, nheads=8, 
        encoder_layers=6, decoder_layers=6, use_pos_encoding=False):
        """Initializes model

        Args:
            model (str): Timm model descriptor for backbone.
            conv_out (int): Backbone output channels.
            num_classes (int): Number of possible label classes
            hidden_dim (int, optional): Hidden channels from linear projection of
            backbone output. Defaults to 256.
            nheads (int, optional): Number of MHA heads. Defaults to 8.
            encoder_layers (int, optional): Number of encoders. Defaults to 6.
            decoder_layers (int, optional): Number of decoders. Defaults to 6.
            use_pos_encoding (bool, optional): Flag for use of position encoding. 
            Defaults to False.
        """        
        
        super().__init__()

        self.num_classes = num_classes
        self.hidden_dim = hidden_dim
        self.use_pos_encoding = use_pos_encoding

        self.backbone = TimmBackbone(model)
        self.conv = nn.Conv2d(conv_out, hidden_dim, 1)
        self.transformer = nn.Transformer(
            hidden_dim, nheads, encoder_layers, decoder_layers)

        if self.use_pos_encoding:
            # returns the encoding object
            self.pos_encoder = PositionalEncodingPermute2D(hidden_dim)

            # returns the summing object
            self.encoding_adder = Summer(self.pos_encoder)

        # prediction head
        self.classifier = nn.Linear(num_classes * hidden_dim, num_classes)

        # learnable label embedding
        self.label_emb = nn.Parameter(torch.rand(1, num_classes, hidden_dim))

    def forward(self, x):
        """Passes batch through network

        Args:
            x (Tensor): Batch of images

        Returns:
            Tensor: Output of classification head
        """        
        # produces output of shape [N x C x H x W]
        out = self.backbone(x)
        
        # reduce number of feature planes for the transformer
        h = self.conv(out)
        B, C, H, W = h.shape

        # add position encodings
        if self.use_pos_encoding:
            
            # input with encoding added
            h = self.encoding_adder(h*0.1)

        # convert h from [N x C x H x W] to [H*W x N x C] (N=batch size)
        # this corresponds to the [SIZE x BATCH_SIZE x EMBED_DIM] dimensions 
        # that the transformer expects
        h = h.flatten(2).permute(2, 0, 1)
        
        # image feature vector "h" is sent in after transformation above; we 
        # also convert label_emb from [1 x TARGET x (hidden)EMBED_SIZE] to 
        # [TARGET x BATCH_SIZE x (hidden)EMBED_SIZE]
        label_emb = self.label_emb.repeat(B, 1, 1)
        label_emb = label_emb.transpose(0, 1)
        h = self.transformer(h, label_emb).transpose(0, 1)
        
        # output from transformer was of dim [TARGET x BATCH_SIZE x EMBED_SIZE];
        # however, we transposed it to [BATCH_SIZE x TARGET x EMBED_SIZE] above.
        # below we reshape to [BATCH_SIZE x TARGET*EMBED_SIZE].
        #
        # next, we project transformer outputs to class labels
        h = torch.reshape(h,(B, self.num_classes * self.hidden_dim))

        return self.classifier(h)

In [9]:
class Query2LabelTrainModule(pl.LightningModule):
    def __init__(
        self,
        data,
        backbone_desc,
        conv_out_dim,
        hidden_dim,
        num_encoders,
        num_decoders,
        num_heads,
        batch_size,
        image_dim,
        learning_rate,
        momentum,
        weight_decay,
        n_classes,
        thresh=0.5,
        use_cutmix=False,
        use_pos_encoding=False,
        loss="BCE",
    ):
        super().__init__()

        # Key parameters
        self.save_hyperparameters(ignore=["model", "data"])
        self.data = data
        self.model = Query2Label(
            model=backbone_desc,
            conv_out=conv_out_dim,
            num_classes=n_classes,
            hidden_dim=hidden_dim,
            nheads=num_heads,
            encoder_layers=num_encoders,
            decoder_layers=num_decoders,
            use_pos_encoding=use_pos_encoding,
        )
        if loss == "BCE":
            self.base_criterion = nn.BCEWithLogitsLoss()
        elif loss == "ASL":
            self.base_criterion = AsymmetricLoss(gamma_neg=1, gamma_pos=0)

        self.criterion = CutMixCriterion(self.base_criterion)

    def forward(self, x):
        x = self.model(x)
        return x

    def evaluate(self, batch, stage=None):
        pass
#         x, y = batch
#         y_hat = self(x)
#         loss = self.base_criterion(y_hat, y.type(torch.float))

#         rmap = tf.retrieval_average_precision(y_hat, y.type(torch.int))

#         category_prec = precision(
#             y_hat,
#             y.type(torch.int),
#             average="macro",
#             num_classes=self.hparams.n_classes,
#             threshold=self.hparams.thresh,
#             task="multiclass",
#         )
#         category_recall = tf.recall(
#             y_hat,
#             y.type(torch.int),
#             average="multiclass",
#             num_classes=self.hparams.n_classes,
#             threshold=self.hparams.thresh,
#             task="multilabel",
#         )
#         category_f1 = tf.f1_score(
#             y_hat,
#             y.type(torch.int),
#             average="multiclass",
#             num_classes=self.hparams.n_classes,
#             threshold=self.hparams.thresh,
#             task="multilabel",
#         )

#         overall_prec = precision(
#             y_hat, y.type(torch.int), threshold=self.hparams.thresh, 
#             task="multiclass"
#         )
#         overall_recall = tf.recall(
#             y_hat, y.type(torch.int), threshold=self.hparams.thresh, 
#             task="multiclass"
#         )
#         overall_f1 = tf.f1_score(
#             y_hat, y.type(torch.int), threshold=self.hparams.thresh, 
#             task="multiclass"
#         )

#         if stage:
#             self.log(f"{stage}_loss", loss, prog_bar=True)
#             self.log(f"{stage}_rmap", rmap, prog_bar=True, on_step=False, on_epoch=True)

#             self.log(f"{stage}_cat_prec", category_prec, prog_bar=True)
#             self.log(f"{stage}_cat_recall", category_recall, prog_bar=True)
#             self.log(f"{stage}_cat_f1", category_f1, prog_bar=True)

#             self.log(f"{stage}_ovr_prec", overall_prec, prog_bar=True)
#             self.log(f"{stage}_ovr_recall", overall_recall, prog_bar=True)
#             self.log(f"{stage}_ovr_f1", overall_f1, prog_bar=True)

#             # log prediction examples to wandb
#             """
#             pred = self.model(x)
#             pred_keys = pred[0].sigmoid().tolist()
#             pred_keys = [0 if p < self.hparams.thresh else 1 for p in pred_keys]


#             mapper = cc.COCOCategorizer()
#             pred_lbl = mapper.get_labels(pred_keys)
            
#             try:
#                 self.logger.experiment.log({"val_pred_examples": [wandb.Image(x[0], caption=pred_lbl)]})
#             except AttributeError:
#                 pass
            # """

    def training_step(self, batch, batch_idx):
        if self.hparams.use_cutmix:
            x, y = batch
            y_hat = self(x)
            # y1, y2, lam = y
            loss = self.criterion(y_hat, y)

        else:
            x, y = batch
            y_hat = self(x)
            loss = self.base_criterion(y_hat, y.type(torch.float))
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def test_step(self, batch, batch_idx):
        self.evaluate(batch, "test")

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(),
            lr=self.hparams.learning_rate,
            betas=(0.9, 0.999),
            weight_decay=self.hparams.weight_decay,
        )

        lr_scheduler_dict = {
            "scheduler": OneCycleLR(
                optimizer,
                self.hparams.learning_rate,
                epochs=self.trainer.max_epochs,
                steps_per_epoch=len(self.data.train_dataloader()),
                anneal_strategy="cos",
            ),
            "interval": "step",
        }
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_dict}
        # return optimizer


In [10]:
pl.seed_everything(42)

Global seed set to 42


42

In [11]:
param_dict = {
    "backbone_desc":"resnet50",
    "conv_out_dim":512,
    "hidden_dim":256,
    "num_encoders":1,
    "num_decoders":2,
    "num_heads":8,
    "batch_size":16,
    "image_dim":224,
    "learning_rate":0.0001, 
    "momentum":0.9,
    "weight_decay":0.01, 
    "n_classes":290,
    "thresh":0.5,
    "use_cutmix":True,
    "use_pos_encoding":False,
    "loss":"ASL"
}

In [12]:
img_data_dir = "../../Data/FathomNet2023/coco_style/"

In [13]:
coco = COCODataModule(
    img_data_dir,
    img_size=param_dict["image_dim"],
    batch_size=param_dict["batch_size"],
    num_workers=24,
    use_cutmix=param_dict["use_cutmix"],
    cutmix_alpha=1.0)
param_dict["data"] = coco

In [14]:
pl_model = Query2LabelTrainModule(**param_dict)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth" to /root/.cache/torch/hub/checkpoints/resnet50_a1_0-14fe96d1.pth


In [15]:
# Comment out if not using wandb
wandb_logger = WandbLogger(
    project="coco-labeling", 
    save_dir="training/logs/",
    log_model=True)
wandb_logger.watch(pl_model, log="all")

trainer = pl.Trainer(
    max_epochs=24,
    precision=16,
    accelerator='gpu', 
    devices=1,
    logger=wandb_logger, # Comment out if not using wandb
    default_root_dir="training/checkpoints/",
    callbacks=[TQDMProgressBar(refresh_rate=10)])
trainer.fit(pl_model, param_dict["data"])

# 11, 249249

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: picekl. Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/usr/local/lib/python3.8/dist-packages/lightning_fabric/connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


NOTE! Installing ujson may make loading annotations faster.
Loading annotations into memory...
Done (t=0.26s)
Creating index...
index created!
Loading annotations into memory...
Done (t=0.03s)
Creating index...
index created!


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:556: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

  | Name           | Type           | Params
--------------------------------------------------
0 | model          | Query2Label    | 49.7 M
1 | base_criterion | AsymmetricLoss | 0     
--------------------------------------------------
49.7 M    Trainable params
0         Non-trainable params
49.7 M    Total params
198.868   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

RuntimeError: Given groups=1, weight of size [256, 512, 1, 1], expected input[16, 2048, 7, 7] to have 512 channels, but got 2048 channels instead